In [ ]:
!wget https://www.dropbox.com/s/r6u59ljhhjdg6j0/negative.csv
!wget https://www.dropbox.com/s/fnpq3z4bcnoktiv/positive.csv 

In [320]:
import pandas as pd
import numpy as np
from keras.preprocessing.text import Tokenizer
import gensim
import pandas as pd
import numpy as np
from itertools import chain
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.models import Model as tfModel
from tensorflow.keras.layers import Dense, Activation, Dropout, LSTM, Bidirectional, TimeDistributed, InputLayer
from tensorflow.keras.layers import Embedding, Conv1D, Conv2D, Input, concatenate, SpatialDropout1D, Flatten
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from tensorflow.keras import optimizers
import nltk
import numpy as np
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from collections import Counter

In [219]:
col_names = ['id', 'tdate', 'tmane', 'ttext', 'ttype', 'trep', 'tfav', 'tstcount', 'tfol', 'tfrien', 'listcount']
pos = pd.read_csv('positive.csv', sep=';', names=col_names, index_col=False)
neg = pd.read_csv('negative.csv', sep=';', names=col_names, index_col=False)

In [220]:
neg['ttype'] = [0 for i in range(len(neg))]

In [221]:
pos = pos[:500]
neg = neg[:500]

In [222]:
corpus = pd.concat([pos, neg])
corpus = corpus.sample(frac=1).reset_index(drop=True)
corpus = corpus[["ttext", "ttype"]]

## 0. Предобработка данных

In [227]:
!pip install ufal.udpipe

In [223]:
import sys
import os
import wget
import re
from ufal.udpipe import Model, Pipeline

"""
Этот скрипт принимает на вход необработанный русский текст 
(одно предложение на строку или один абзац на строку).
Он токенизируется, лемматизируется и размечается по частям речи с использованием UDPipe.
На выход подаётся последовательность разделенных пробелами лемм с частями речи 
("зеленый_NOUN трамвай_NOUN").
Их можно непосредственно использовать в моделях с RusVectōrēs (https://rusvectores.org).
Примеры запуска:
echo 'Мама мыла раму.' | python3 rus_preprocessing_udpipe.py
zcat large_corpus.txt.gz | python3 rus_preprocessing_udpipe.py | gzip > processed_corpus.txt.gz
"""

def num_replace(word):
    newtoken = "x" * len(word)
    return newtoken


def clean_token(token, misc):
    """
    :param token:  токен (строка)
    :param misc:  содержимое поля "MISC" в CONLLU (строка)
    :return: очищенный токен (строка)
    """
    out_token = token.strip().replace(" ", "")
    if token == "Файл" and "SpaceAfter=No" in misc:
        return None
    return out_token


def clean_lemma(lemma, pos):
    """
    :param lemma: лемма (строка)
    :param pos: часть речи (строка)
    :return: очищенная лемма (строка)
    """
    out_lemma = lemma.strip().replace(" ", "").replace("_", "").lower()
    if "|" in out_lemma or out_lemma.endswith(".jpg") or out_lemma.endswith(".png"):
        return None
    if pos != "PUNCT":
        if out_lemma.startswith("«") or out_lemma.startswith("»"):
            out_lemma = "".join(out_lemma[1:])
        if out_lemma.endswith("«") or out_lemma.endswith("»"):
            out_lemma = "".join(out_lemma[:-1])
        if (
            out_lemma.endswith("!")
            or out_lemma.endswith("?")
            or out_lemma.endswith(",")
            or out_lemma.endswith(".")
        ):
            out_lemma = "".join(out_lemma[:-1])
    return out_lemma


def list_replace(search, replacement, text):
    search = [el for el in search if el in text]
    for c in search:
        text = text.replace(c, replacement)
    return text


def unify_sym(text):  # принимает строку в юникоде
    text = list_replace(
        "\u00AB\u00BB\u2039\u203A\u201E\u201A\u201C\u201F\u2018\u201B\u201D\u2019",
        "\u0022",
        text,
    )

    text = list_replace(
        "\u2012\u2013\u2014\u2015\u203E\u0305\u00AF", "\u2003\u002D\u002D\u2003", text
    )

    text = list_replace("\u2010\u2011", "\u002D", text)

    text = list_replace(
        "\u2000\u2001\u2002\u2004\u2005\u2006\u2007\u2008\u2009\u200A\u200B\u202F\u205F\u2060\u3000",
        "\u2002",
        text,
    )

    text = re.sub("\u2003\u2003", "\u2003", text)
    text = re.sub("\t\t", "\t", text)

    text = list_replace(
        "\u02CC\u0307\u0323\u2022\u2023\u2043\u204C\u204D\u2219\u25E6\u00B7\u00D7\u22C5\u2219\u2062",
        ".",
        text,
    )

    text = list_replace("\u2217", "\u002A", text)

    text = list_replace("…", "...", text)

    text = list_replace("\u2241\u224B\u2E2F\u0483", "\u223D", text)

    text = list_replace("\u00C4", "A", text)  # латинская
    text = list_replace("\u00E4", "a", text)
    text = list_replace("\u00CB", "E", text)
    text = list_replace("\u00EB", "e", text)
    text = list_replace("\u1E26", "H", text)
    text = list_replace("\u1E27", "h", text)
    text = list_replace("\u00CF", "I", text)
    text = list_replace("\u00EF", "i", text)
    text = list_replace("\u00D6", "O", text)
    text = list_replace("\u00F6", "o", text)
    text = list_replace("\u00DC", "U", text)
    text = list_replace("\u00FC", "u", text)
    text = list_replace("\u0178", "Y", text)
    text = list_replace("\u00FF", "y", text)
    text = list_replace("\u00DF", "s", text)
    text = list_replace("\u1E9E", "S", text)

    currencies = list(
        "\u20BD\u0024\u00A3\u20A4\u20AC\u20AA\u2133\u20BE\u00A2\u058F\u0BF9\u20BC\u20A1\u20A0\u20B4\u20A7\u20B0\u20BF\u20A3\u060B\u0E3F\u20A9\u20B4\u20B2\u0192\u20AB\u00A5\u20AD\u20A1\u20BA\u20A6\u20B1\uFDFC\u17DB\u20B9\u20A8\u20B5\u09F3\u20B8\u20AE\u0192"
    )

    alphabet = list(
        '\t\n\r абвгдеёзжийклмнопрстуфхцчшщьыъэюяАБВГДЕЁЗЖИЙКЛМНОПРСТУФХЦЧШЩЬЫЪЭЮЯ,.[]{}()=+-−*&^%$#@!?~;:0123456789§/\|"abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ '
    )

    alphabet.append("'")

    allowed = set(currencies + alphabet)

    cleaned_text = [sym for sym in text if sym in allowed]
    cleaned_text = "".join(cleaned_text)

    return cleaned_text


def process(pipeline, text="Строка", keep_pos=True, keep_punct=False):
    # Если частеречные тэги не нужны (например, их нет в модели), выставьте pos=False
    # в этом случае на выход будут поданы только леммы
    # По умолчанию знаки пунктуации вырезаются. Чтобы сохранить их, выставьте punct=True

    entities = {"PROPN"}
    named = False
    memory = []
    mem_case = None
    mem_number = None
    tagged_propn = []

    # обрабатываем текст, получаем результат в формате conllu:
    processed = pipeline.process(text)

    # пропускаем строки со служебной информацией:
    content = [line for line in processed.split("\n") if not line.startswith("#")]

    # извлекаем из обработанного текста леммы, тэги и морфологические характеристики
    tagged = [w.split("\t") for w in content if w]

    for t in tagged:
        if len(t) != 10:
            continue
        (word_id, token, lemma, pos, xpos, feats, head, deprel, deps, misc) = t
        token = clean_token(token, misc)
        lemma = clean_lemma(lemma, pos)
        if not lemma or not token:
            continue
        if pos in entities:
            if "|" not in feats:
                tagged_propn.append("%s_%s" % (lemma, pos))
                continue
            morph = {el.split("=")[0]: el.split("=")[1] for el in feats.split("|")}
            if "Case" not in morph or "Number" not in morph:
                tagged_propn.append("%s_%s" % (lemma, pos))
                continue
            if not named:
                named = True
                mem_case = morph["Case"]
                mem_number = morph["Number"]
            if morph["Case"] == mem_case and morph["Number"] == mem_number:
                memory.append(lemma)
                if "SpacesAfter=\\n" in misc or "SpacesAfter=\s\\n" in misc:
                    named = False
                    past_lemma = "::".join(memory)
                    memory = []
                    tagged_propn.append(past_lemma + "_PROPN")
            else:
                named = False
                past_lemma = "::".join(memory)
                memory = []
                tagged_propn.append(past_lemma + "_PROPN")
                tagged_propn.append("%s_%s" % (lemma, pos))
        else:
            if not named:
                if (
                    pos == "NUM" and token.isdigit()
                ):  # Заменяем числа на xxxxx той же длины
                    lemma = num_replace(token)
                tagged_propn.append("%s_%s" % (lemma, pos))
            else:
                named = False
                past_lemma = "::".join(memory)
                memory = []
                tagged_propn.append(past_lemma + "_PROPN")
                tagged_propn.append("%s_%s" % (lemma, pos))

    if not keep_punct:
        tagged_propn = [word for word in tagged_propn if word.split("_")[1] != "PUNCT"]
    if not keep_pos:
        tagged_propn = [word.split("_")[0] for word in tagged_propn]
    return tagged_propn


# URL of the UDPipe model
udpipe_model_url = "https://rusvectores.org/static/models/udpipe_syntagrus.model"
udpipe_filename = udpipe_model_url.split("/")[-1]

if not os.path.isfile(udpipe_filename):
    print("UDPipe model not found. Downloading...", file=sys.stderr)
    wget.download(udpipe_model_url)

print("\nLoading the model...", file=sys.stderr)
model = Model.load(udpipe_filename)
process_pipeline = Pipeline(
    model, "tokenize", Pipeline.DEFAULT, Pipeline.DEFAULT, "conllu"
)

print("Processing input...", file=sys.stderr)
for input_line in sys.stdin:
    res = unify_sym(input_line.strip())
    output = process(process_pipeline, text=res)
    print(" ".join(output))


Loading the model...
Processing input...


In [224]:
output = process(process_pipeline, text=corpus["ttext"][1])
#print(" ".join(output))
print(output)

['rt_PROPN', '@radughniykot_X', 'вечеринка_NOUN', 'привет_NOUN', 'о_ADP', 'хотя_SCONJ', 'ты_PRON', 'насрать_VERB', 'x_NUM', 'xxx_NUM']


In [225]:
corpus["ttext_pos"] = [" ".join(process(process_pipeline, text=twit)) for twit in corpus["ttext"]]

In [229]:
corpus["ttext_processed"] = [" ".join(process(process_pipeline, keep_pos=False, text=twit)) for twit in corpus["ttext"]]

#1. Простая модель, embedding слой обучается внутри модели

In [233]:
from sklearn.model_selection import train_test_split
y = np.array(list(corpus['ttype']))
X = np.array(list(corpus["ttext_processed"]))
twit_train,twit_test,tag_train,tag_test = train_test_split(X,y,test_size=0.2)

In [236]:
from collections import Counter

vocab = Counter()
for sent in twit_train:
    sent = [word.lower() for word in sent.split()]
    vocab.update(sent)

In [237]:
filtered_vocab = {word for word in vocab if vocab[word] > 5}

In [238]:
len(filtered_vocab)

192

In [239]:
word2id = {word:i + 2 for i,word in enumerate(filtered_vocab)}
word2id['pad'] = 0
word2id['unk'] = 1  

id2word = {i:word for word, i in word2id.items()}

In [240]:
chars = set([letter for word in filtered_vocab for letter in word])
n_chars = len(chars)
char2id = {c: i + 2 for i, c in enumerate(chars)}
char2id["pad"] = 0
char2id["unk"] = 1

id2char = {i:char for char, i in char2id.items()}

In [241]:
char_max_len = max(len(x) for x in filtered_vocab)
print("максимальная длина слова:", char_max_len)

максимальная длина слова: 11


In [242]:
def data2ints(data, smth2id):
  int_data = []
  for seq in data:
      int_seq = []
      for i in seq:
        int_seq.append(smth2id.get(i.lower(), 1))
  
      int_data.append(int_seq)
  return int_data

In [243]:
X_train_ids, X_test_ids = data2ints(twit_train, word2id), data2ints(twit_test, word2id)
y_train_ids, y_test_ids = list(tag_train), list(tag_test)

In [244]:
sent_max_len = max(len(x) for x in X_train_ids)
print("максимальная длина твита:", sent_max_len)

максимальная длина твита: 144


In [245]:
from tensorflow.keras.preprocessing.sequence import pad_sequences

X_train, X_test = pad_sequences(X_train_ids, maxlen=sent_max_len, padding='post'), pad_sequences(X_test_ids, maxlen=sent_max_len, padding='post')

In [246]:
tag_test = np.array(list(tag_test))
tag_train = np.array(list(tag_train))

In [247]:
def make_X_char(sentences):
  X_char = []
  for sentence in sentences:
      sent_seq = []
      for i in range(sent_max_len):
          word_seq = []
          for j in range(char_max_len):
              try:
                  word_seq.append(char2id[sentence[i][j].lower()])
              except:
                  word_seq.append(char2id["pad"])
          sent_seq.append(word_seq)
      X_char.append(np.array(sent_seq))
  return np.array(X_char)

In [248]:
X_char_train, X_char_test = make_X_char(twit_train), make_X_char(twit_test)

In [249]:
tag_train, tag_test = to_categorical(tag_train, num_classes=2), to_categorical(tag_test, num_classes=2)

In [250]:
len(word2id)

194

In [254]:
word_in = Input(shape=(sent_max_len,))
emb_word = Embedding(input_dim=len(word2id), output_dim=20, input_length=sent_max_len)(word_in)
#lstm1 = Bidirectional(LSTM(units=128, return_sequences=True, recurrent_dropout=0.2))(emb_word)
lstm2 = Bidirectional(LSTM(units=128, recurrent_dropout=0.2))(emb_word)
out = Dense(2, activation="softmax")(lstm2)


model = tfModel(inputs= word_in, outputs=out)

optimizer = optimizers.Adam(learning_rate=0.001)
model.compile(optimizer=optimizer, loss="binary_crossentropy", metrics = [tf.keras.metrics.Precision(), tf.keras.metrics.Recall(), tf.keras.metrics.Accuracy()])

In [255]:
emb_word

<KerasTensor: shape=(None, 144, 20) dtype=float32 (created by layer 'embedding_20')>

In [256]:
model.summary()

Model: "model_18"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_21 (InputLayer)        [(None, 144)]             0         
_________________________________________________________________
embedding_20 (Embedding)     (None, 144, 20)           3880      
_________________________________________________________________
bidirectional_20 (Bidirectio (None, 256)               152576    
_________________________________________________________________
dense_23 (Dense)             (None, 2)                 514       
Total params: 156,970
Trainable params: 156,970
Non-trainable params: 0
_________________________________________________________________


In [ ]:
model.fit(X_train, tag_train, validation_data=(X_test, tag_test), batch_size=128, epochs=30, verbose=1)

#2. Простая модель, подгружаются обученные эмбеддинги для русского языка

In [ ]:
pip install wget

In [258]:
from gensim.test.utils import common_texts
from gensim.models import Word2Vec as w2v
import wget

#>>> model = Word2Vec(sentences=common_texts, vector_size=100, window=5, min_count=1, workers=4)
#>>> model.save("word2vec.model")

In [259]:
import zipfile
model_url = 'http://vectors.nlpl.eu/repository/11/180.zip'
m = wget.download(model_url)
model_file = model_url.split('/')[-1]
with zipfile.ZipFile(model_file, 'r') as archive:
    stream = archive.open('model.bin')
    model = gensim.models.KeyedVectors.load_word2vec_format(stream, binary=True)

In [260]:
model.wv.get_keras_embedding(train_embeddings=False)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  """Entry point for launching an IPython kernel.


In [278]:
y = np.array(list(corpus['ttype']))
X = np.array(list(corpus["ttext_pos"]))
twit_train,twit_test,tag_train,tag_test = train_test_split(X,y,test_size=0.2)

from collections import Counter

vocab = Counter()
for sent in twit_train:
    sent = [word for word in sent.split()]
    vocab.update(sent)

filtered_vocab = {word for word in vocab if vocab[word] > 5}

In [286]:
model['песня_NOUN'].shape

(300,)

In [294]:
from keras.preprocessing.text import Tokenizer
import gensim
import pandas as pd
import numpy as np
from itertools import chain

#w2v = gensim.models.Word2Vec.load("models/w2v.model")
vocab = model.wv.vocab    
t = Tokenizer()

filtered_vocab_list = list(filtered_vocab)

vocab_size = len(filtered_vocab_list) + 1
t.fit_on_texts(filtered_vocab_list)

def get_weight_matrix():
    # define weight matrix dimensions with all 0
    weight_matrix = np.zeros((vocab_size, model.vector_size))
    # step vocab, store vectors using the Tokenizer's integer mapping
    for i in range(len(filtered_vocab)):
        try:
            weight_matrix[i + 1] = model[filtered_vocab_list[i]]
        except:
            continue
    return weight_matrix

weight_matrix = get_weight_matrix()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:8: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  


In [290]:
model.get_keras_embedding(train_embeddings=False)

In [299]:
emb_word

In [306]:
word_in = Input(shape=(sent_max_len,))

#emb_word = model.get_keras_embedding(train_embeddings=False)(word_in)
emb_word = tf.keras.layers.Embedding(input_dim=len(word2id), output_dim=300, trainable=False, weights=[weight_matrix])(word_in)
#emb_word = Embedding(input_dim=len(word2id), output_dim=20, input_length=sent_max_len)(word_in)
#lstm1 = Bidirectional(LSTM(units=128, return_sequences=True, recurrent_dropout=0.2))(emb_word)
lstm2 = Bidirectional(LSTM(units=128, recurrent_dropout=0.2))(emb_word)
out = Dense(2, activation="softmax")(lstm2)


model2 = tfModel(inputs= word_in, outputs=out)

optimizer = optimizers.Adam(learning_rate=0.001)
model2.compile(optimizer=optimizer, loss="binary_crossentropy", metrics = [tf.keras.metrics.Precision(), tf.keras.metrics.Recall(), tf.keras.metrics.Accuracy()])

In [309]:
tag_train
tag_train, tag_test = to_categorical(tag_train, num_classes=2), to_categorical(tag_test, num_classes=2)

In [311]:
tf.config.run_functions_eagerly(True)

In [312]:
model2.fit(X_train, tag_train, validation_data=(X_test, tag_test), batch_size=128, epochs=30, verbose=1)

/usr/local/lib/python3.6/dist-packages/tensorflow/python/data/ops/dataset_ops.py:3504: UserWarning: Even though the tf.config.experimental_run_functions_eagerly option is set, this option does not apply to tf.data functions. tf.data functions are still traced and executed as graphs.
  "Even though the tf.config.experimental_run_functions_eagerly "


Epoch 1/30
7/7 [==============================] - 32s 4s/step - loss: 0.7050 - precision_20: 0.5013 - recall_20: 0.5013 - accuracy: 0.0000e+00 - val_loss: 0.6927 - val_precision_20: 0.5050 - val_recall_20: 0.5050 - val_accuracy: 0.0000e+00
Epoch 2/30
7/7 [==============================] - 31s 4s/step - loss: 0.6894 - precision_20: 0.5325 - recall_20: 0.5325 - accuracy: 0.0000e+00 - val_loss: 0.6936 - val_precision_20: 0.5400 - val_recall_20: 0.5400 - val_accuracy: 0.0000e+00
Epoch 3/30
7/7 [==============================] - 31s 4s/step - loss: 0.6817 - precision_20: 0.5600 - recall_20: 0.5600 - accuracy: 0.0000e+00 - val_loss: 0.6914 - val_precision_20: 0.4800 - val_recall_20: 0.4800 - val_accuracy: 0.0000e+00
Epoch 4/30
7/7 [==============================] - 31s 5s/step - loss: 0.6772 - precision_20: 0.5738 - recall_20: 0.5738 - accuracy: 0.0000e+00 - val_loss: 0.6924 - val_precision_20: 0.4950 - val_recall_20: 0.4950 - val_accuracy: 0.0000e+00
Epoch 5/30
7/7 [========================

#3. Простая модель, fasttext эмбеддинги обучаются на всем корпусе с нуля

In [ ]:
from gensim.models import FastText
#model = FastText(size=4, window=3, min_count=1, sentences=twit_train, epochs=10)
ft = gensim.models.FastText(twit_train, size=20, iter=20)

In [ ]:
max_len = 140

tokenizer = tf.keras.preprocessing.text.Tokenizer(lower=True)
tokenizer.fit_on_texts(twit_train)

sequence_docs = tokenizer.texts_to_sequences(twit_train)
sequence_docs = tf.keras.preprocessing.sequence.pad_sequences(sequence_docs, maxlen=max_len)

# extract fasttext learned embedding and put them in a numpy array

embedding_matrix_ft = np.random.random((len(tokenizer.word_index) + 1, ft.vector_size))

pas = 0
for word,i in tokenizer.word_index.items():
    
    try:
        embedding_matrix_ft[i] = ft.wv[word]
    except:
        pas+=1

# define a keras model and load the pretrained fasttext weights matrix

inp = Input(shape=(max_len,))
emb = Embedding(len(tokenizer.word_index) + 1, ft.vector_size, 
                weights=[embedding_matrix_ft], trainable=False)(inp)

In [ ]:
embedding_matrix_ft.shape

(1212, 20)

In [ ]:
emb_word

<KerasTensor: shape=(None, 140, 20) dtype=float32 (created by layer 'embedding_12')>

In [ ]:
word_in = Input(shape=(sent_max_len, ))
#emb_word = Embedding(input_dim=len(word2id), output_dim=20, mask_zero=True)(word_in)
emb_word = Embedding(len(tokenizer.word_index) + 1, ft.vector_size, 
                weights=[embedding_matrix_ft], trainable=False)(word_in)
#lstm1 = Bidirectional(LSTM(units=128, return_sequences=True, recurrent_dropout=0.2))(emb_word)
lstm2 = Bidirectional(LSTM(units=128, recurrent_dropout=0.2))(emb_word)
out = Dense(1, activation="softmax")(lstm2)

model = Model(inputs= word_in, outputs=out)

optimizer = optimizers.Adam(learning_rate=0.001)
model.compile(optimizer=optimizer, loss="binary_crossentropy", metrics = [tf.keras.metrics.Precision(), tf.keras.metrics.Recall(), tf.keras.metrics.Accuracy()])

In [ ]:
unseen_docs = twit_test
unseen_docs = [d.lower().split() for d in unseen_docs]

sequence_unseen_docs = tokenizer.texts_to_sequences(unseen_docs)
X_test = tf.keras.preprocessing.sequence.pad_sequences(sequence_unseen_docs, maxlen=max_len)

In [ ]:
model.fit(sequence_docs, tag_train, validation_data=(X_test, tag_test), batch_size=128, epochs=30, verbose=1)

Epoch 1/30
2/2 [==============================] - 7s 2s/step - loss: 0.6946 - precision_4: 0.4865 - recall_4: 1.0000 - accuracy: 0.4865 - val_loss: 0.7035 - val_precision_4: 0.5500 - val_recall_4: 1.0000 - val_accuracy: 0.5500
Epoch 2/30
2/2 [==============================] - 2s 703ms/step - loss: 0.6947 - precision_4: 0.4969 - recall_4: 1.0000 - accuracy: 0.4969 - val_loss: 0.6870 - val_precision_4: 0.5500 - val_recall_4: 1.0000 - val_accuracy: 0.5500
Epoch 3/30
2/2 [==============================] - 2s 679ms/step - loss: 0.6884 - precision_4: 0.4865 - recall_4: 1.0000 - accuracy: 0.4865 - val_loss: 0.7098 - val_precision_4: 0.5500 - val_recall_4: 1.0000 - val_accuracy: 0.5500
Epoch 4/30
2/2 [==============================] - 2s 713ms/step - loss: 0.6836 - precision_4: 0.4839 - recall_4: 1.0000 - accuracy: 0.4839 - val_loss: 0.7237 - val_precision_4: 0.5500 - val_recall_4: 1.0000 - val_accuracy: 0.5500
Epoch 5/30
2/2 [==============================] - 2s 701ms/step - loss: 0.6822 - pr

#4. Усложненная архитектура с CharCNN

In [313]:
word_in = Input(shape=(sent_max_len,))

#emb_word = model.get_keras_embedding(train_embeddings=False)(word_in)
emb_word = tf.keras.layers.Embedding(input_dim=len(word2id), output_dim=300, trainable=False, weights=[weight_matrix])(word_in)
#emb_word = Embedding(input_dim=len(word2id), output_dim=20, input_length=sent_max_len)(word_in)
#lstm1 = Bidirectional(LSTM(units=128, return_sequences=True, recurrent_dropout=0.2))(emb_word)
lstm2 = Bidirectional(LSTM(units=128, recurrent_dropout=0.2))(emb_word)
out = Dense(2, activation="softmax")(lstm2)


model2 = tfModel(inputs= word_in, outputs=out)

optimizer = optimizers.Adam(learning_rate=0.001)
model2.compile(optimizer=optimizer, loss="binary_crossentropy", metrics = [tf.keras.metrics.Precision(), tf.keras.metrics.Recall(), tf.keras.metrics.Accuracy()])

In [315]:
# один вход для слов
word_in = Input(shape=(sent_max_len))
emb_word = tf.keras.layers.Embedding(input_dim=len(word2id), output_dim=300, trainable=False, weights=[weight_matrix])(word_in)

# другой вход для символов
char_in = Input(shape=(sent_max_len, char_max_len))
emb_char = TimeDistributed(Embedding(input_dim=len(char2id), output_dim=10, input_length=char_max_len))(char_in)
# свертка на символах (CharRNN) применяется к каждому слову отдельно
char_enc = TimeDistributed(Conv1D(filters=12, kernel_size=3))(emb_char)
char_flat = TimeDistributed(Flatten())(char_enc)

# LSTM проходится по всей последовательности, на каждом шаге беря эмбеддинг слова по словарю + символьный эмбеддинг слова 
x = concatenate([emb_word, char_flat])
main_lstm = Bidirectional(LSTM(units=128, recurrent_dropout=0.2))(x)

out = Dense(2, activation="softmax")(main_lstm)

model3 = tfModel(inputs=[char_in, word_in], outputs=out)


optimizer = optimizers.Adam(learning_rate=0.001)
model3.compile(optimizer=optimizer, loss="binary_crossentropy", metrics=[tf.keras.metrics.Precision(), tf.keras.metrics.Recall(), tf.keras.metrics.Accuracy()])


In [316]:
model3.fit([X_char_train, X_train], tag_train, validation_data=([X_char_test, X_test], tag_test), batch_size=128, epochs=30, verbose=1)

/usr/local/lib/python3.6/dist-packages/tensorflow/python/data/ops/dataset_ops.py:3504: UserWarning: Even though the tf.config.experimental_run_functions_eagerly option is set, this option does not apply to tf.data functions. tf.data functions are still traced and executed as graphs.
  "Even though the tf.config.experimental_run_functions_eagerly "


Epoch 1/30
7/7 [==============================] - 39s 5s/step - loss: 0.7043 - precision_22: 0.4801 - recall_22: 0.4801 - accuracy: 0.0000e+00 - val_loss: 0.6964 - val_precision_22: 0.5300 - val_recall_22: 0.5300 - val_accuracy: 0.0000e+00
Epoch 2/30
7/7 [==============================] - 37s 5s/step - loss: 0.6877 - precision_22: 0.5528 - recall_22: 0.5528 - accuracy: 0.0000e+00 - val_loss: 0.6982 - val_precision_22: 0.5400 - val_recall_22: 0.5400 - val_accuracy: 0.0000e+00
Epoch 3/30
7/7 [==============================] - 37s 5s/step - loss: 0.6825 - precision_22: 0.5426 - recall_22: 0.5426 - accuracy: 0.0000e+00 - val_loss: 0.7052 - val_precision_22: 0.5200 - val_recall_22: 0.5200 - val_accuracy: 0.0000e+00
Epoch 4/30
7/7 [==============================] - 37s 5s/step - loss: 0.6876 - precision_22: 0.5543 - recall_22: 0.5543 - accuracy: 0.0000e+00 - val_loss: 0.7015 - val_precision_22: 0.5550 - val_recall_22: 0.5550 - val_accuracy: 0.0000e+00
Epoch 5/30
7/7 [========================

# 5. Выводы и эксперементы с гиперпараметрами

На примере простой модели видно, что лучше всего работает тот вариант модели, где мы используем предобученные эмбеддинги для слов. Что логично, по сути мы используем дополнительные данные («знания» предобученных эмбеддингов о языке вне текущего корпуса) и поэтому лучше учимся работать на неизвестных данных. 

Усложненная модель работает лучше, чем простая – что, в целом, тоже логично. Для сравнения моделей между собой я смотрю на precision и recall (accuracy плохо считается внутри обучения, я не стала это переделвать). В нашем случае, когда модель работает лучше, обе эти метрики выше.

Что касается экспериментов с гиперпараметрами, судя по тому, как падает loss, нашей модели ещё далеко до переобучения, и мы можем просто увеличить количество эпох. Действительно, увеличения количество эпох с 30 до 35 даёт заметный прирост в качестве модели.

In [333]:
# один вход для слов
word_in = Input(shape=(sent_max_len))
emb_word = tf.keras.layers.Embedding(input_dim=len(word2id), output_dim=300, trainable=False, weights=[weight_matrix])(word_in)

# другой вход для символов
char_in = Input(shape=(sent_max_len, char_max_len))
emb_char = TimeDistributed(Embedding(input_dim=len(char2id), output_dim=10, input_length=char_max_len))(char_in)
# свертка на символах (CharRNN) применяется к каждому слову отдельно
char_enc = TimeDistributed(Conv1D(filters=12, kernel_size=3))(emb_char)

char_flat = TimeDistributed(Flatten())(char_enc)

# LSTM проходится по всей последовательности, на каждом шаге беря эмбеддинг слова по словарю + символьный эмбеддинг слова 
x = concatenate([emb_word, char_flat])
main_lstm = Bidirectional(LSTM(units=128, recurrent_dropout=0.2))(x)

out = Dense(2, activation="softmax")(main_lstm)

model4 = tfModel(inputs=[char_in, word_in], outputs=out)


optimizer = optimizers.Adam(learning_rate=0.001)
model4.compile(optimizer=optimizer, loss="binary_crossentropy", metrics=[tf.keras.metrics.Precision(), tf.keras.metrics.Recall(), tf.keras.metrics.Accuracy()])


In [334]:
model4.fit([X_char_train, X_train], tag_train, validation_data=([X_char_test, X_test], tag_test), batch_size=128, epochs=35, verbose=1)

/usr/local/lib/python3.6/dist-packages/tensorflow/python/data/ops/dataset_ops.py:3504: UserWarning: Even though the tf.config.experimental_run_functions_eagerly option is set, this option does not apply to tf.data functions. tf.data functions are still traced and executed as graphs.
  "Even though the tf.config.experimental_run_functions_eagerly "


Epoch 1/35
7/7 [==============================] - 37s 5s/step - loss: 0.7020 - precision_28: 0.4787 - recall_28: 0.4787 - accuracy: 0.0000e+00 - val_loss: 0.6926 - val_precision_28: 0.5200 - val_recall_28: 0.5200 - val_accuracy: 0.0000e+00
Epoch 2/35
7/7 [==============================] - 37s 5s/step - loss: 0.6914 - precision_28: 0.5505 - recall_28: 0.5505 - accuracy: 0.0000e+00 - val_loss: 0.6900 - val_precision_28: 0.5500 - val_recall_28: 0.5500 - val_accuracy: 0.0000e+00
Epoch 3/35
7/7 [==============================] - 37s 5s/step - loss: 0.6839 - precision_28: 0.5441 - recall_28: 0.5441 - accuracy: 0.0000e+00 - val_loss: 0.6918 - val_precision_28: 0.5500 - val_recall_28: 0.5500 - val_accuracy: 0.0000e+00
Epoch 4/35
7/7 [==============================] - 37s 5s/step - loss: 0.6754 - precision_28: 0.5836 - recall_28: 0.5836 - accuracy: 0.0000e+00 - val_loss: 0.6969 - val_precision_28: 0.5400 - val_recall_28: 0.5400 - val_accuracy: 0.0000e+00
Epoch 5/35
7/7 [========================